# Doodles

Exploring the dataset.

In [ ]:
import pandas as pd

df = pd.read_csv("IW_os_placenames.csv")
df.head()

,ID,NAMES_URI,NAME1,NAME1_LANG,NAME2,NAME2_LANG,TYPE,LOCAL_TYPE,GEOMETRY_X,GEOMETRY_Y,...,COUNTY_UNITARY,COUNTY_UNITARY_URI,COUNTY_UNITARY_TYPE,REGION,REGION_URI,COUNTRY,COUNTRY_URI,RELATED_SPATIAL_OBJECT,SAME_AS_DBPEDIA,SAME_AS_GEONAMES
0,SU22.csv:osgb4000000074770417,http://data.ordnancesurvey.co.uk/id/4000000074...,Isle of Wight Hill,NaN,NaN,NaN,landform,Hill Or Mountain,424844,137337,...,Hampshire,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,NaN,NaN
1,SU86.csv:osgb4000000074808919,http://data.ordnancesurvey.co.uk/id/4000000074...,Isle of Wight Pond,NaN,NaN,NaN,hydrography,Inland Water,495262,172358,...,Windsor and Maidenhead,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,NaN,NaN
2,SZ06.csv:osgb4000000026226540,http://data.ordnancesurvey.co.uk/id/4000000026...,Isle of Wight Road,NaN,NaN,NaN,transportNetwork,Named Road,403311,77620,...,Dorset,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South West,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,NaN,NaN
3,SZ28.csv:osgb4000000074551734,http://data.ordnancesurvey.co.uk/id/4000000074...,Bouldnor,NaN,NaN,NaN,populatedPlace,Village,436404,89600,...,Isle of Wight,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,http://dbpedia.org/resource/Bouldnor,NaN
4,SZ28.csv:osgb4000000074551705,http://data.ordnancesurvey.co.uk/id/4000000074...,Freshwater,NaN,NaN,NaN,populatedPlace,Village,433576,87130,...,Isle of Wight,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,"http://dbpedia.org/resource/Freshwater,_Isle_o...",http://sws.geonames.org/2649069


In [4]:
# Use: COUNTY_UNITARY_URI
# http://data.ordnancesurvey.co.uk/id/7000000000025469

In [8]:
df["TYPE"].unique(), df["LOCAL_TYPE"].unique()

(array(['landform', 'hydrography', 'transportNetwork', 'populatedPlace',
        'other', 'landcover'], dtype=object),
 array(['Hill Or Mountain', 'Inland Water', 'Named Road', 'Village',
        'Suburban Area', 'Hamlet', 'Town', 'Postcode',
        'Section Of Named Road', 'Numbered Road', 'Woodland Or Forest',
        'Other Landcover', 'Valley', 'Coastal Headland',
        'Other Coastal Landform', 'Wetland', 'Island', 'Cliff Or Slope',
        'Bay', 'Harbour', 'Medical Care Accommodation',
        'Electricity Production', 'Primary Education',
        'Vehicular Ferry Terminal', 'Section Of Numbered Road',
        'Spot Height', 'Tidal Water', 'Secondary Education',
        'Special Needs Education', 'Other Settlement', 'Other Landform',
        'Channel', 'Beach', 'Estuary', 'Sea', 'Railway',
        'Passenger Ferry Terminal',
        'Non State Primary Education,Non State Secondary Education',
        'Railway Station', 'Bus Station', 'Airport', 'Further Education',
        'H

## PDF Explorer

In [1]:
import fitz  # PyMuPDF
import requests
from PIL import Image
import io
import cv2
import numpy as np
from collections import defaultdict

def download_pdf_if_needed(source):
    """Download PDF from URL if needed, or return local path"""
    if source.startswith(("http://", "https://")):
        # It's a URL, download it
        filename = "downloaded_document.pdf"
        print(f"Downloading PDF from {source}...")
        response = requests.get(source)
        response.raise_for_status()

        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"PDF saved as {filename}")
        return filename
    else:
        # It's a local file path
        import os

        if not os.path.exists(source):
            raise FileNotFoundError(f"Local PDF file not found: {source}")
        print(f"Using local PDF file: {source}")
        return source


def method1_detect_scanned_vs_multiple_images(pdf_path):
    """Method 1: Detect pages with multiple images (illustrations + text scan)"""
    doc = fitz.open(pdf_path)
    pages_with_multiple_images = []

    print("Method 1: Checking for pages with multiple images...")

    for page_num in range(len(doc)):
        page = doc[page_num]
        image_list = page.get_images()

        # For scanned docs, look for pages with MORE than just the main scan
        if len(image_list) > 1:
            print(
                f"Page {page_num + 1}: Found {len(image_list)} images (likely has separate illustrations)"
            )
            pages_with_multiple_images.append(
                {"page": page_num + 1, "image_count": len(image_list)}
            )
        elif len(image_list) == 1:
            # Check if the single image is unusually large (might contain illustrations)
            img = image_list[0]
            # Get image dimensions if possible
            try:
                pix = fitz.Pixmap(doc, img[0])
                if pix.width * pix.height > 2000000:  # Large image threshold
                    print(
                        f"Page {page_num + 1}: Large single image ({pix.width}x{pix.height}) - may contain illustrations"
                    )
                    pages_with_multiple_images.append(
                        {
                            "page": page_num + 1,
                            "image_count": 1,
                            "note": "large_single_image",
                        }
                    )
                pix = None
            except:
                pass

    doc.close()
    return pages_with_multiple_images


def method2_analyze_drawings(pdf_path):
    """Method 2: Analyze vector graphics and drawing commands"""
    doc = fitz.open(pdf_path)
    pages_with_drawings = []

    print("\nMethod 2: Analyzing drawing commands...")

    for page_num in range(len(doc)):
        page = doc[page_num]

        # Get drawing commands
        drawings = page.get_drawings()

        # Filter out simple lines/borders (keep complex shapes)
        complex_drawings = [d for d in drawings if len(d.get("items", [])) > 2]

        if complex_drawings:
            print(
                f"Page {page_num + 1}: Found {len(complex_drawings)} complex drawing(s)"
            )
            pages_with_drawings.append(
                {"page": page_num + 1, "drawing_count": len(complex_drawings)}
            )

    doc.close()
    return pages_with_drawings


def method3_visual_analysis_for_scans(pdf_path, edge_threshold=0.05):
    """Method 3: Analyze scanned pages for illustration characteristics"""
    doc = fitz.open(pdf_path)
    pages_with_illustrations = []

    print("\nMethod 3: Analyzing scanned pages for illustration patterns...")

    for page_num in range(len(doc)):
        page = doc[page_num]

        # Render page as image
        mat = fitz.Matrix(1.5, 1.5)  # Higher resolution for better analysis
        pix = page.get_pixmap(matrix=mat)
        img_data = pix.tobytes("png")

        # Convert to PIL Image then to OpenCV format
        try:
            pil_img = Image.open(io.BytesIO(img_data))
            # Convert PIL to RGB if it's not already, then to numpy array
            if pil_img.mode != "RGB":
                pil_img = pil_img.convert("RGB")

            # Convert to numpy array and ensure it's the right type
            img_array = np.array(pil_img)
            if img_array.dtype != np.uint8:
                img_array = img_array.astype(np.uint8)

            # Convert to grayscale
            if len(img_array.shape) == 3:
                cv_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            else:
                cv_img = img_array  # Already grayscale

        except Exception as e:
            print(f"Error processing page {page_num + 1}: {e}")
            continue

        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(cv_img, (5, 5), 0)

        # Detect edges (illustrations typically have more defined edges)
        edges = cv2.Canny(blurred, 30, 100)
        edge_density = np.sum(edges > 0) / edges.size

        # Look for large connected components (illustrations vs scattered text)
        contours, _ = cv2.findContours(
            edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        large_contours = [c for c in contours if cv2.contourArea(c) > 500]

        # Calculate histogram to detect regions with different tonal characteristics
        hist = cv2.calcHist([blurred], [0], None, [256], [0, 256])
        # Illustrations often have more varied tonal ranges
        hist_variance = np.var(hist)

        # Detect potential illustration characteristics
        has_illustration_features = (
            edge_density > edge_threshold
            or len(large_contours) > 5
            or hist_variance > 1000000
        )

        if has_illustration_features:
            print(
                f"Page {page_num + 1}: Illustration features detected (edges: {edge_density:.3f}, large shapes: {len(large_contours)}, tonal variance: {hist_variance:.0f})"
            )
            pages_with_illustrations.append(
                {
                    "page": page_num + 1,
                    "edge_density": edge_density,
                    "large_contours": len(large_contours),
                    "tonal_variance": hist_variance,
                }
            )

    doc.close()
    return pages_with_illustrations


def method4_text_detection_ocr_alternative(pdf_path):
    """Method 4: Detect text regions using image processing (OCR alternative)"""
    doc = fitz.open(pdf_path)
    pages_likely_illustrations = []

    print("\nMethod 4: Detecting text vs illustration regions...")

    for page_num in range(len(doc)):
        page = doc[page_num]

        # Render page as image
        mat = fitz.Matrix(2.0, 2.0)  # Higher resolution for text detection
        pix = page.get_pixmap(matrix=mat)
        img_data = pix.tobytes("png")

        # Convert to PIL Image then to OpenCV format
        try:
            pil_img = Image.open(io.BytesIO(img_data))
            # Convert PIL to RGB if it's not already, then to numpy array
            if pil_img.mode != "RGB":
                pil_img = pil_img.convert("RGB")

            # Convert to numpy array and ensure it's the right type
            img_array = np.array(pil_img)
            if img_array.dtype != np.uint8:
                img_array = img_array.astype(np.uint8)

            # Convert to grayscale
            if len(img_array.shape) == 3:
                cv_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            else:
                cv_img = img_array  # Already grayscale

        except Exception as e:
            print(f"Error processing page {page_num + 1}: {e}")
            continue

        # Create binary image
        _, binary = cv2.threshold(cv_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Detect horizontal and vertical lines (typical of text layout)
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))

        horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel)

        # Count text-like patterns (small rectangular regions)
        text_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        text_regions = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, text_kernel)

        contours, _ = cv2.findContours(
            text_regions, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        # Count small, text-like contours vs large illustration-like contours
        text_like_contours = 0
        illustration_like_contours = 0

        for contour in contours:
            area = cv2.contourArea(contour)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / h if h > 0 else 0

            # Text characteristics: small, rectangular regions
            if 50 < area < 2000 and 0.2 < aspect_ratio < 5:
                text_like_contours += 1
            # Illustration characteristics: larger, varied shapes
            elif area > 2000:
                illustration_like_contours += 1

        # Calculate ratios
        total_contours = text_like_contours + illustration_like_contours
        if total_contours > 0:
            illustration_ratio = illustration_like_contours / total_contours
        else:
            illustration_ratio = 0

        # Pages with high illustration ratio or very few text-like features
        if illustration_ratio > 0.3 or (total_contours > 0 and text_like_contours < 10):
            print(
                f"Page {page_num + 1}: Likely illustration (ill. ratio: {illustration_ratio:.2f}, text regions: {text_like_contours})"
            )
            pages_likely_illustrations.append(
                {
                    "page": page_num + 1,
                    "illustration_ratio": illustration_ratio,
                    "text_like_regions": text_like_contours,
                    "illustration_like_regions": illustration_like_contours,
                }
            )

    doc.close()
    return pages_likely_illustrations


def analyze_pdf_illustrations(pdf_source):
    """Main function to analyze PDF for illustrations using multiple methods

    Args:
        pdf_source: Either a URL (starting with http/https) or local file path
    """

    # Get the PDF file (download if URL, validate if local path)
    pdf_path = download_pdf_if_needed(pdf_source)

    print(f"\n{'='*60}")
    print("ANALYZING PDF FOR ILLUSTRATIONS")
    print(f"{'='*60}")

    # Method 1: Multiple images or unusually large images
    #multiple_images = method1_detect_scanned_vs_multiple_images(pdf_path)

    # Method 2: Vector drawings (still relevant for some scanned docs)
    #vector_drawings = method2_analyze_drawings(pdf_path)

    # Method 3: Visual analysis optimized for scanned pages
    visual_analysis = method3_visual_analysis_for_scans(pdf_path)

    # Method 4: Text detection using image processing
    text_analysis = method4_text_detection_ocr_alternative(pdf_path)

    # Combine results
    all_pages = set()
    for result_set in [
        multiple_images,
        vector_drawings,
        visual_analysis,
        text_analysis,
    ]:
        all_pages.update(item["page"] for item in result_set)

    print(f"\n{'='*60}")
    print("SUMMARY")
    print(f"{'='*60}")
    print(f"Pages likely containing illustrations: {sorted(all_pages)}")
    print(f"Total pages with potential illustrations: {len(all_pages)}")

    # Detailed breakdown
    print(f"\nDetailed breakdown:")
    print(f"- Multiple/large images: {len(multiple_images)} pages")
    print(f"- Vector drawings: {len(vector_drawings)} pages")
    print(f"- Illustration visual patterns: {len(visual_analysis)} pages")
    print(f"- Low text density: {len(text_analysis)} pages")

    return {
        "multiple_images": multiple_images,
        "vector_drawings": vector_drawings,
        "visual_analysis": visual_analysis,
        "text_analysis": text_analysis,
        "all_illustration_pages": sorted(all_pages),
    }

In [2]:
pdf = "./bim_eighteenth-century_a-tour-to-the-isle-of-wi_tomkins-charles_1796_1.pdf"  

# Use either URL or local path
results = analyze_pdf_illustrations(pdf)
# results = analyze_pdf_illustrations(pdf_path)  # For local file

# Optional: Save detailed results
import json

with open("illustration_analysis.json", "w") as f:
    json.dump(results, f, indent=2)
print("\nDetailed results saved to 'illustration_analysis.json'")

Using local PDF file: ./bim_eighteenth-century_a-tour-to-the-isle-of-wi_tomkins-charles_1796_1.pdf

ANALYZING PDF FOR ILLUSTRATIONS

Method 3: Analyzing scanned pages for illustration patterns...
Page 1: Illustration features detected (edges: 0.025, large shapes: 1, tonal variance: 73884384)
Page 2: Illustration features detected (edges: 0.013, large shapes: 1, tonal variance: 102004648)
Page 3: Illustration features detected (edges: 0.025, large shapes: 0, tonal variance: 83167880)
Page 4: Illustration features detected (edges: 0.009, large shapes: 0, tonal variance: 121493504)
Page 5: Illustration features detected (edges: 0.039, large shapes: 3, tonal variance: 71179120)
Page 6: Illustration features detected (edges: 0.037, large shapes: 2, tonal variance: 69703624)
Page 7: Illustration features detected (edges: 0.029, large shapes: 0, tonal variance: 61343472)
Page 8: Illustration features detected (edges: 0.026, large shapes: 0, tonal variance: 57909032)
Page 9: Illustration featu

KeyboardInterrupt: 

In [ ]:
#%pip install --upgrade opencv-python

  Using cached numpy-2.3.0-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 9.4 MB/s eta 0:00:0000:0100:01m
Using cached numpy-2.3.0-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-diffusion-sdkit 2.1.5 requires einops==0.3.0, but you have einops 0.8.0 which is incompatible.
stable-diffusion-sdkit 2.1.5 requires opencv-python==4.6.0.66, but you have opencv-python 4.11.0.86 which is incompatible.
stable-diffusion-sdkit 2.1.5 requires transformers==4.33.2, but you have transformers 4.46.3 which is incompatible.
langchain-community 0.0.38 requires numpy<2,>=1, but you hav